In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

### Data load

In [35]:
src_folder = os.path.join('data', 'idealista')

# df is created
ds = pd.DataFrame()

for file in os.listdir(src_folder):
    file = os.path.join(src_folder, file)
    ds = pd.concat([ds, pd.read_csv(file)], ignore_index = True).drop_duplicates()

ds = ds.reset_index(drop = True)
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           695 non-null    int64  
 1   title        695 non-null    object 
 2   n_rooms      675 non-null    float64
 3   size         695 non-null    int64  
 4   description  683 non-null    object 
 5   extra_info   689 non-null    object 
 6   image        682 non-null    object 
 7   img_no       695 non-null    int64  
 8   price        695 non-null    int64  
 9   price_form   695 non-null    object 
dtypes: float64(1), int64(4), object(5)
memory usage: 54.4+ KB


In [36]:
cols = ds.columns
cols_exclude = []

#convert int columns to float
for col in cols:
    if ds[col].dtype == 'int64':
        ds[col] = ds[col].astype('float64')
    elif ds[col].dtype == 'object':
        cols_exclude.append(col)

cols_model = [col for col in cols if col not in cols_exclude]

### Deal with NAs

In [37]:
ds['n_rooms'] = ds['n_rooms'].fillna(1)

### Data split

In [38]:
x = ds[cols_model].drop(['price'], axis=1)
y_original = ds['price']
y = y_original/ds['size']

# slit data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

### Model

In [39]:
model = LinearRegression()
model.fit(x_train, y_train)
r2_scored = model.score(x_test, y_test)
print("R2 Scored: ", r2_scored)

R2 Scored:  0.07387563200620362


### Evaluate model

y is price/m2

In [40]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print('Training RMSE score: {}.'.format(rmse_train))
print('Testing RMSE score: {}\n'.format(rmse_test))

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)  
print('Training MAE score: {}'.format(mae_train))
print('Testing MAE score: {}'.format(mae_test))

Training RMSE score: 4.479106203912407.
Testing RMSE score: 5.460754956200045

Training MAE score: 2.467289533039443
Testing MAE score: 2.5558483083292876


y is price

In [41]:
y_train_pred = model.predict(x_train)*x_train['size']
y_test_pred = model.predict(x_test)*x_test['size']

rmse_train = np.sqrt(mean_squared_error(y_original[y_train.index], y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_original[y_test.index], y_test_pred))
#print rmse and mae
print('Training RMSE score: {}.'.format(rmse_train))
print('Testing RMSE score: {}\n'.format(rmse_test))

mae_train = mean_absolute_error(y_original[y_train.index], y_train_pred)
mae_test = mean_absolute_error(y_original[y_test.index], y_test_pred)  
print('Training MAE score: {}'.format(mae_train))
print('Testing MAE score: {}'.format(mae_test))

Training RMSE score: 413.01224856446424.
Testing RMSE score: 515.0433081027282

Training MAE score: 248.20052766444184
Testing MAE score: 270.6304902103753
